# Lab 1: Silver Fund Quant Data Module and Returns

In this lab we will:
- Explore how to pull data from the Silver Fund Quant data module.
- Demonstrate the different properties of returns.

## Setup

In order to have a smooth experience with this lab do the following:

### 1. Log into the Fulton Super Computer.

In order to log into you must have an account at [https://rc.byu.edu/](https://rc.byu.edu/) and be added to the `grp_quant` group by Brian Boyer.

It can take some time to get approved so make sure to create an account and reach out to Brian promptly.

### 2. Clone this repo to the desired location (I prefer to have a `Projects` folder where I keep all of my repositories).

Clone the repo by running
```bash
git clone https://github.com/BYUSilverFund/sf-quant-labs.git
```

### 3. Install `uv` (Package Manager)

We use `uv` to create and manage virtual environments.

To install `uv` run

```bash
curl -LsSf https://astral.sh/uv/install.sh | sh
```

Check that `uv` is installed by running

```bash
uv --version
```

If this returns an error you might need to add uv to your path. Run:

```bash
source $HOME/.local/bin/env
```

Restart your terminal for the changes to take effect.

### 4. Create a Virtual Environment

The virtual environment will make it so that we have consistent package and Python versions across all devices.

With `uv` it is really easy to create a virtual environment with synced dependencies.

Just run

```bash
uv sync
```

Activate the environment by running

``` bash
source .venv/bin/activate
```

## Imports

With all of the setup out of the way we will import the necessary Python packages for the lab.

- `sf_quant`: Silver Fund Quant Team package that includes modules for loading data, optimizing portfolios, backtesting, and analyzing performance.
- `datetime`: Native Python library for creating Python `date` types.
- `polars`: Data frame library similar to Pandas but with a much cleaner API and 100x speed ups.

In [1]:
import sf_quant as sf
import polars as pl
import datetime as dt

/home/stiten/Documents/sf-quant-labs/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-07 22:39:53,285	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Data

Use the following code to pull data for our investment universe from 2024-01-01 to 2024-12-31.

In [2]:
print(sf.data.get_assets_columns()) # lists available columns

shape: (31, 2)
┌─────────────────────────────────┬─────────┐
│ column                          ┆ dtype   │
│ ---                             ┆ ---     │
│ str                             ┆ str     │
╞═════════════════════════════════╪═════════╡
│ date                            ┆ Date    │
│ rootid                          ┆ String  │
│ barrid                          ┆ String  │
│ issuerid                        ┆ String  │
│ instrument                      ┆ String  │
│ name                            ┆ String  │
│ cusip                           ┆ String  │
│ ticker                          ┆ String  │
│ price                           ┆ Float64 │
│ return                          ┆ Float64 │
│ specific_return                 ┆ Float64 │
│ market_cap                      ┆ Float64 │
│ price_source                    ┆ String  │
│ currency                        ┆ String  │
│ iso_country_code                ┆ String  │
│ iso_currency_code               ┆ String  │
│ yield            

In [3]:
start = dt.date(2024, 1, 1)
end =  dt.date(2024, 12, 31)

columns = [
    'date',
    'barrid',
    'return'
    
]

df = sf.data.load_assets(
    start=start,
    end=end,
    in_universe=True,
    columns=columns
)

df

date,barrid,return
date,str,f64
2024-01-02,"""USA06Z1""",-10.2623
2024-01-03,"""USA06Z1""",-1.2071
2024-01-04,"""USA06Z1""",-0.1929
2024-01-05,"""USA06Z1""",0.5155
2024-01-08,"""USA06Z1""",5.3846
…,…,…
2024-12-24,"""USBQOR1""",2.5872
2024-12-26,"""USBQOR1""",4.293
2024-12-27,"""USBQOR1""",-5.108


## Log returns

### Instructions
1. Compute the log returns for each asset.
2. Compute the cummulative log returns for each asset.
3. Run the assertion cell to make sure you're results are correct.

Make sure to sort prior to computing time series metrics and use `.over()` apply the computation in groups.

Log returns have the nice property of being additive. Use this to your advantage!

In [4]:
def task_compute_log_returns(df: pl.DataFrame) -> pl.DataFrame:
    """
    Compute the log returns for each security and date combo.

    Args:
        df (pl.DataFrame): Data frame containing columns date, barrid, and return

    Returns:
        pl.DataFrame: Data frame containing columns date, barrid, return, and log_return
    """

    df = df.with_columns(
        pl.col("return")
            .truediv(100)   
            .log1p()   
            .alias("log_return")
    )

    return df

df_log = task_compute_log_returns(df)

df_log

date,barrid,return,log_return
date,str,f64,f64
2024-01-02,"""USA06Z1""",-10.2623,-0.108279
2024-01-03,"""USA06Z1""",-1.2071,-0.012144
2024-01-04,"""USA06Z1""",-0.1929,-0.001931
2024-01-05,"""USA06Z1""",0.5155,0.005142
2024-01-08,"""USA06Z1""",5.3846,0.052446
…,…,…,…
2024-12-24,"""USBQOR1""",2.5872,0.025543
2024-12-26,"""USBQOR1""",4.293,0.042034
2024-12-27,"""USBQOR1""",-5.108,-0.052431


In [5]:
def task_compute_cumulative_log_returns(df_log: pl.DataFrame) -> pl.DataFrame:
    """
    Compute the cummulative log returns for each security and date combo.

    Args:
        df_log (pl.DataFrame): Data frame containing columns date, barrid, return, and log_return

    Returns:
        pl.DataFrame: Data frame containing columns date, barrid, return, log_return, and cumulative_log_return
    """

    df = df_log.sort(["barrid", "date"]).with_columns(
        pl.col("log_return").cum_sum().over("barrid").alias("cumulative_log_return")
    )

    return df

df_cum_log = task_compute_cumulative_log_returns(df_log)

df_cum_log

date,barrid,return,log_return,cumulative_log_return
date,str,f64,f64,f64
2024-01-02,"""USA06Z1""",-10.2623,-0.108279,-0.108279
2024-01-03,"""USA06Z1""",-1.2071,-0.012144,-0.120424
2024-01-04,"""USA06Z1""",-0.1929,-0.001931,-0.122355
2024-01-05,"""USA06Z1""",0.5155,0.005142,-0.117213
2024-01-08,"""USA06Z1""",5.3846,0.052446,-0.064766
…,…,…,…,…
2024-12-24,"""USBQOR1""",2.5872,0.025543,0.087605
2024-12-26,"""USBQOR1""",4.293,0.042034,0.129639
2024-12-27,"""USBQOR1""",-5.108,-0.052431,0.077208


In [6]:
assert df_cum_log['cumulative_log_return'].max() == 2.8475532093020557

## Compounded Returns

### Instructions

1. Compute the cumulative compounded returns for each asset.
2. Run the assertion to check that your results are correct.

In [7]:
def task_compute_cumulative_compounded_returns(df_cum_log: pl.DataFrame) -> pl.DataFrame:
    """
    Compute the cumulative compounded returns for each security.

    Args:
        df_cum_log (pl.DataFrame): Data frame containing columns date, barrid, return, log_return, and cumulative_log_return

    Returns:
        pl.DataFrame: Data frame containing columns date, barrid, return, log_return, cumulative_log_return, and cumulative_compouned_return
    """

    df_cum_comp = df_cum_log.sort(["barrid", "date"]).with_columns(
        (pl.col("return").truediv(100).add(1).cum_prod().over("barrid") - 1).alias("cumulative_compounded_return")
    )
    return df_cum_comp

df_cum_comp = task_compute_cumulative_compounded_returns(df_cum_log)

df_cum_comp

date,barrid,return,log_return,cumulative_log_return,cumulative_compounded_return
date,str,f64,f64,f64,f64
2024-01-02,"""USA06Z1""",-10.2623,-0.108279,-0.108279,-0.102623
2024-01-03,"""USA06Z1""",-1.2071,-0.012144,-0.120424,-0.113455
2024-01-04,"""USA06Z1""",-0.1929,-0.001931,-0.122355,-0.115165
2024-01-05,"""USA06Z1""",0.5155,0.005142,-0.117213,-0.110604
2024-01-08,"""USA06Z1""",5.3846,0.052446,-0.064766,-0.062714
…,…,…,…,…,…
2024-12-24,"""USBQOR1""",2.5872,0.025543,0.087605,0.091556
2024-12-26,"""USBQOR1""",4.293,0.042034,0.129639,0.138417
2024-12-27,"""USBQOR1""",-5.108,-0.052431,0.077208,0.080267


In [8]:
assert df_cum_comp['cumulative_compounded_return'].max() == 16.245533963705515

## Exponentiation

Note that the max cumulative log return is different from the cumulative compounded return.

Why is that?

The answer is that the cumulative log return is still in log space!

### Instructions

1. Exponentiate the cumulative log returns to put them back into the original space.
2. Check that the exponentiated returns match the cumulative compounded returns.

In [9]:
df_cum_comp

date,barrid,return,log_return,cumulative_log_return,cumulative_compounded_return
date,str,f64,f64,f64,f64
2024-01-02,"""USA06Z1""",-10.2623,-0.108279,-0.108279,-0.102623
2024-01-03,"""USA06Z1""",-1.2071,-0.012144,-0.120424,-0.113455
2024-01-04,"""USA06Z1""",-0.1929,-0.001931,-0.122355,-0.115165
2024-01-05,"""USA06Z1""",0.5155,0.005142,-0.117213,-0.110604
2024-01-08,"""USA06Z1""",5.3846,0.052446,-0.064766,-0.062714
…,…,…,…,…,…
2024-12-24,"""USBQOR1""",2.5872,0.025543,0.087605,0.091556
2024-12-26,"""USBQOR1""",4.293,0.042034,0.129639,0.138417
2024-12-27,"""USBQOR1""",-5.108,-0.052431,0.077208,0.080267


In [10]:
def task_exponentiate_returns(df_cum_comp: pl.DataFrame) -> pl.DataFrame:
    """
    Exponentiate the cumulative log returns.

    Args:
        df_cum_comp: Data frame containing date, barrid, return, log_return, cumulative_log_return, and cumulative_compouned_return.

    Returns:
        pl.DataFrame: Data frame containing all previous columns plus exponentiated_returns
    """

    df_exp_ret = df_cum_comp.with_columns(
        (pl.col("cumulative_log_return").exp() - 1).alias("exponentiated_return")
    )
    return df_exp_ret
df_exp = task_exponentiate_returns(df_cum_comp)

df_exp

date,barrid,return,log_return,cumulative_log_return,cumulative_compounded_return,exponentiated_return
date,str,f64,f64,f64,f64,f64
2024-01-02,"""USA06Z1""",-10.2623,-0.108279,-0.108279,-0.102623,-0.102623
2024-01-03,"""USA06Z1""",-1.2071,-0.012144,-0.120424,-0.113455,-0.113455
2024-01-04,"""USA06Z1""",-0.1929,-0.001931,-0.122355,-0.115165,-0.115165
2024-01-05,"""USA06Z1""",0.5155,0.005142,-0.117213,-0.110604,-0.110604
2024-01-08,"""USA06Z1""",5.3846,0.052446,-0.064766,-0.062714,-0.062714
…,…,…,…,…,…,…
2024-12-24,"""USBQOR1""",2.5872,0.025543,0.087605,0.091556,0.091556
2024-12-26,"""USBQOR1""",4.293,0.042034,0.129639,0.138417,0.138417
2024-12-27,"""USBQOR1""",-5.108,-0.052431,0.077208,0.080267,0.080267


In [11]:
assert df_exp['cumulative_compounded_return'].max() == df_exp['exponentiated_return'].max()